# Face Recognition in Videos  

## Install / Import

In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566171 sha256=3ca1d3e9c8637df2aef3108159db0131f2d12cf782f489638ee7d69d6bcc7cee
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import face_recognition
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import time
import numpy as np
import pickle

## Load encodings

In [ ]:
pickle_name = "face_encodings_celebs.pickle"
data_encoding = pickle.loads(open(pickle_name, "rb").read())
list_encodings = data_encoding["encodings"]
list_names = data_encoding["names"]

## Helper functions

In [ ]:
max_width = 900 # leave None if you don't want to resize

In [ ]:
def resize_video(width, height, max_width = 600):
  if (width > max_width):
    proportion = width / height
    video_width = max_width
    video_height = int(video_width / proportion)
  else:
    video_width = width
    video_height = height

  return video_width, video_height

In [ ]:
resizing = 0.5

In [ ]:
def recognize_faces(image, list_encodings, list_names, resizing=0.25, tolerance=0.6):
  image = cv2.resize(image, (0, 0), fx=resizing, fy=resizing)

  img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  face_locations = face_recognition.face_locations(img_rgb)
  face_encodings = face_recognition.face_encodings(img_rgb, face_locations)

  face_names = []
  conf_values = []
  for encoding in face_encodings:
    matches = face_recognition.compare_faces(list_encodings, encoding, tolerance=tolerance)
    name = "Not identified"

    face_distances = face_recognition.face_distance(list_encodings, encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
      name = list_names[best_match_index]
    face_names.append(name)
    conf_values.append(face_distances[best_match_index])

  face_locations = np.array(face_locations)
  # we are scalig back up the face locations, since the frame we detected was scaled by the scaleFactor of `resizing` variable
  face_locations = face_locations / resizing
  return face_locations.astype(int), face_names, conf_values

In [ ]:
def show_recognition(frame, face_locations, face_names, conf_values):

  for face_loc, name, conf in zip(face_locations, face_names, conf_values):
    y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

    conf = "{:.8f}".format(conf)
    cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 0.7, (20, 255, 0), 2, lineType=cv2.LINE_AA)
    cv2.rectangle(frame, (x1, y1), (x2, y2), (20, 255, 0), 4)
    if name != "Not identified":
        cv2.putText(frame, conf,(x1, y2 + 15), cv2.FONT_HERSHEY_DUPLEX, 0.5, (20, 255, 0), 1, lineType=cv2.LINE_AA)

  return frame

## Video Configurations

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = 24

## Reading the video file

In [ ]:
video_file = "video04.mp4"
result_file = 'result.avi'

cap = cv2.VideoCapture(video_file)
connected, video = cap.read()
video_width, video_height = video.shape[1], video.shape[0]

if max_width is not None:
  video_width, video_height = resize_video(video_width, video_height, max_width)
print(video_width, video_height)

video_output = cv2.VideoWriter(result_file, fourcc, fps, (video_width, video_height))

900 507


## Processing the video

In [ ]:
frames_show = 20
current_frame = 1
max_frames = -1

In [ ]:
while (cv2.waitKey(1) < 0):
  connected, frame = cap.read()

  if not connected:
    break

  if max_frames > -1 and current_frame > max_frames:
      break

  (H, W) = frame.shape[:2]

  t = time.time()

  if max_width is not None:
    frame = cv2.resize(frame, (video_width, video_height))

  face_locations, face_names, conf_values = recognize_faces(frame, list_encodings, list_names, resizing)
  frame = show_recognition(frame, face_locations, face_names, conf_values)

  cv2.putText(frame, " frame processed in {:.2f} seconds".format(time.time() - t), (20, video_height-20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (250, 250, 250), 0, lineType=cv2.LINE_AA)  # cv2.LINE_AA to improve the quality of the text

  video_output.write(frame)

  if current_frame <= frames_show:
    cv2_imshow(cv2.resize(frame, (0,0), fx=0.75, fy=0.75))

  current_frame = current_frame + 1

print("Finished")
video_output.release()
cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!ffmpeg -y -loglevel panic -i result.avi result.mp4

In [ ]:
def show_video(name_file, width=700, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(name_file, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="Video" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [ ]:
show_video("result.mp4")

Output hidden; open in https://colab.research.google.com to view.